In [1]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [2]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [3]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [4]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [5]:
# !fc-cache -f

In [6]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [7]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [465]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [466]:
%%capture pwd
!pwd

In [467]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('why-did-the-bank-of-england-need-a-review-of-its-forecasting-record',
 300,
 500)

In [468]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [469]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [470]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                            if c in i['encoding'][x]['axis']:
                                i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [489]:
df = pd.read_csv("raw/f1.csv",skiprows=1).dropna(how='all',axis=1)
df.columns=[i.replace('.','').lower() for i in df.columns]
df=df.rename(columns={'x':'x0','y':'y0'})

In [490]:
df['x0']+=0.5
df['x0']=df['x0'][0]+0.07+(df['x0']-df['x0'][0])*0.93
for i in range(1,5):
    df['x'+str(i)]=df['x'+str(i)][0]+1.22+(df['x'+str(i)]-df['x'+str(i)][0])*0.66
    df['y'+str(i)]-=2.6

In [491]:
dfs=[]
for i in range(5):
    df['x'+str(i)+'t']=pd.to_datetime(df['x'+str(i)].dropna().astype(str).str.split('.').str[0]+'-01-01')+\
    pd.to_timedelta((('0.'+df['x'+str(i)].dropna().astype(str).str.split('.').str[1]).astype(float)*365.25).round().astype(int).astype(str)+'D')
    dfs.append(df.set_index('x'+str(i)+'t')[['y'+str(i)]].resample('1M').mean().interpolate())
df=pd.concat(dfs,axis=1).reset_index().rename(columns={'index':'x'})
df['ym']=df['y4']+(df['y1']-df['y4'])/2
df['y2b']=2*df['ym']-df['y2']
df['y3b']=2*df['ym']-df['y3']

In [492]:
readme, f, fc = save(df,"fig1_cpi",LOCAL)

,x,y0,y1,y2,y3,y4,ym,y2b,y3b
0,2017-01-31,2.064516,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-02-28,2.301075,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-31,2.537634,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-04-30,2.774194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-05-31,2.782258,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
xmin='2017-01-01'
xmax='2025-01-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':2},{'x':xmax,'y':2}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='x:T')
xaxis2= alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x="x:T",y='y:Q')
line1=base.mark_line(color=colors['eco-dot'],interpolate='basis').encode(y='y0:Q').transform_filter('datum.y0>0')
area1=base.mark_area(color=colors['eco-dot'],interpolate='basis',opacity=0.2).encode(y='y4:Q',y2='y1:Q')
area2=base.mark_area(color=colors['eco-dot'],interpolate='basis',opacity=0.2).encode(y='y2:Q',y2='y2b:Q')
area3=base.mark_area(color=colors['eco-dot'],interpolate='basis',opacity=0.2).encode(y='y3:Q',y2='y3b:Q')
points= alt.Chart(pd.DataFrame([{'x':'2022-10-15','y':9.6,'t':'October 2022'},
                                {'x':'2023-01-15','y':8.8,'t':'January 2023'}]))\
    .mark_circle(color=colors["eco-gray"]).encode(x="x:T",y='y:Q')
labels1= points.mark_text(color=colors["eco-gray"],align='right',dx=-10).encode(x="x:T",y='y:Q',text='t:N')
labels2= points.mark_text(color=colors["eco-gray"],align='left',dx=10,dy=1).encode(x="x:T",y='y:Q',text='t2:N').transform_calculate(t2='toString(datum.y)+"%"')

layer1 = (
    (xaxis+points+xaxis2+line1+area3+area2+area1+labels1+labels2).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 10, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)
# layer1.save("visualisation/" + f + ".json")
    
if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

# Fig 2

In [ ]:
df = pd.read_excel("raw/MonthlyUKMacroDataset.xlsx")
df['date']=pd.to_datetime(df['Unnamed: 0'].str.replace('(','-').str.replace(')','-01'))
df=df[['date','D12Ura','D12LYa','D12LPa','D12LWa']]
df=df.set_index('date').stack().reset_index()\
    .replace('D12LPa','Annual UK inflation')\
    .replace('D12LYa','Annual change in UK GDP')\
    .replace('D12Ura','Annual change in UK employment')\
    .replace('D12LWa','Annual change in UK wages')
df.columns=['date','ind','value']

In [ ]:
readme, f, fc = save(df,"fig2_ons",LOCAL)

In [ ]:
xmin='2000-03-01'
xmax='2024-03-01'
xaxis = alt.Chart(fc).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1,clip=True).encode(
    alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
).transform_calculate(y='0')

line1=xaxis.mark_line(color=colors['eco-dot'],interpolate='basis',clip=True).encode(y='value:Q')

layer1 = (
    (xaxis+line1).properties(height=180, width=250).facet(facet=alt.Facet('ind:N', 
    title=None,header={'labelColor':colors["eco-gray"],'labelFontSize':11}),columns=2,
    resolve={'scale':{'y':'independent'}})
)

layer1.save("visualisation/" + f + "_no_branding.json")
# if DARK:
#     layer2=dark(f+'_no_branding')
#     layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    # if DARK:
    #     layer2.save("visualisation/" + f + "_no_branding_dark.svg")
    #     layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 6, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0)\
    .encode(x=alt.X('x:T',axis=None),y=alt.Y('y:Q',axis=None),url='img:N')\
    .properties(height=0, width=50)
layer1=alt.hconcat(layer1,logo).configure(font='Circular Std Book').configure_view(stroke=None)

layer1.save("visualisation/" + f + ".json")
# if DARK:
#     layer2=dark(f)
#     layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    # if DARK:
    #     layer2.save("visualisation/" + f + "_dark.svg")
    #     layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
# if DARK:
#     layer2.display()

# Fig 3

## a

In [ ]:
df = pd.read_excel("raw/UKEnergy.xlsx")
df=df.rename(columns={'Unnamed: 0':'Year','passthrough%':'passthrough'})
df['time']=pd.to_datetime(df['Year'].astype(str)+'-01-01')

In [ ]:
readme, f, fc = save(df,"fig3a_pt",LOCAL)

In [ ]:
xmin='1860-01-01'
xmax='2025-01-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Mtoe (million tonnes of oil equivalent)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
# yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
# ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
# ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 16-24'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='time:T')
line1=base.mark_line(color=colors['eco-dot']).encode(y='TotalFuel:Q').transform_filter('datum.TotalFuel>0')
line2=base.mark_line(color=colors['eco-turquiose']).encode(y='UKTotOil:Q').transform_filter('datum.UKTotOil>0')
line3=base.mark_line(color=colors['eco-purple']).encode(y='UKGas:Q').transform_filter('datum.UKGas>0')
label1=line1.mark_text(color=colors['eco-dot'],text='Total Fuel',align='left',dx=5).transform_filter('datum.Year==2021')
label2=line2.mark_text(color=colors['eco-turquiose'],text='Oil',align='left',dx=5).transform_filter('datum.Year==2021')
label3=line3.mark_text(color=colors['eco-purple'],text='Natural Gas',align='left',dx=5).transform_filter('datum.Year==2021')

layer1 = (
    (xaxis+line1+line3+line2+label1+label2+label3).properties(height=300, width=400, title={'text':'A','color':colors['eco-gray']})
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 250, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

## b

In [ ]:
readme, f, fc = save(df,"fig3b_pt2",LOCAL)

In [ ]:
base = alt.Chart(fc).encode(    alt.X(
        "Dp:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="Δp",
            titleY=-15,
            titleX=387,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "passthrough:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="pass through %",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
                           )
points=base.mark_circle().encode(color=alt.Color('Year:Q',
    scale=alt.Scale(range=['white',colors['eco-light-blue'],colors['eco-blue']]),
    legend=alt.Legend(title="Year",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            tickCount=8,
                      padding=30,
            format='.0f')))

labels=points.mark_text(align='left',dx=5,dy=3).encode(text='Year:N')\
    .transform_filter(alt.FieldOneOfPredicate(field='Year', 
    oneOf=[1922,1923,1879,1973,2022,2023,1973,1975,2020,2021,1976,1980,1933,2007,2008]))

layer1 = (
    (points+labels).properties(height=300, width=400, title={'text':'B','color':colors['eco-gray']})
    .configure(font='Circular Std Book').configure_view(stroke=None)
)


layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": 0.25, "y": 100, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

# Fig 4

## Clean up `SVG`s
`base64 URI` encode images

In [ ]:
from base64 import b64encode

In [ ]:
from os import listdir
from os.path import isfile, join
mypath='./visualisation/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
svgs=[i for i in onlyfiles if i[-4:]=='.svg']

In [ ]:
for svg in svgs:
    s=open(mypath+svg,'r').read()
    si=s.find('"image mark" xlink:href=')
    if si>-1:
        s2=s[si+25:]
        imgurl=s2[:s2.find('"')]
        if 'data:image/png;' not in imgurl:
            s1=s[:si+25]+s2.replace(imgurl,'data:image/png;base64,'+b64encode(requests.get(imgurl).content).decode('utf-8'))
            open(mypath+svg,'w').write(s1)
            print(svg)

## README

In [ ]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))